In [1]:
from dotenv import load_dotenv
from langchain_groq import ChatGroq

load_dotenv()

model = "qwen/qwen3-32b"

llm = ChatGroq(model = model, temperature = 0.0)

When we talk about agents, a significant part of an "agent" is simple code logic,
iteratively rerunning LLM calls and processing their output. The exact logic varies
significantly, but one well-known example is the **ReAct** agent.

![ReAct process](https://www.aurelio.ai/_next/image?url=%2Fimages%2Fposts%2Fai-agents%2Fai-agents-00.png&w=640&q=75)

**Re**ason + **Act**ion (ReAct) agents use iterative _reasoning_ and _action_ steps to
incorporate chain-of-thought and tool-use into their execution. During the _reasoning_
step, the LLM generates the steps to take to answer the query. Next, the LLM generates
the _action_ input, which our code logic parses into a tool call.

![Agentic graph of ReAct](https://www.aurelio.ai/_next/image?url=%2Fimages%2Fposts%2Fai-agents%2Fai-agents-01.png&w=640&q=75)

Following our action step, we get an observation from the tool call. Then, we feed the
observation back into the agent executor logic for a final answer or further reasoning
and action steps.

The agent and agent executor we will be building will follow this pattern.

In [2]:
# Tools.
from langchain_core.tools import tool

@tool
def add(x: float, y:float) -> float:
    """Add x and y"""

    return x+y

@tool
def subtract(x: float, y: float) -> float:
    """Subtract x from y"""

    return y-x

@tool
def multiply(x: float, y: float) -> float:
    """Multiply x and y"""

    return x*y

@tool
def exponentiate(x: float, y: float) -> float:
    """Raise x to the power of y"""

    return x**y

In [3]:
add

StructuredTool(name='add', description='Add x and y', args_schema=<class 'langchain_core.utils.pydantic.add'>, func=<function add at 0x000002400C811DC0>)

In [4]:
print(f"{add.name=}\n{add.description=}")

add.name='add'
add.description='Add x and y'


In [5]:
exponentiate.args_schema.model_json_schema()

{'description': 'Raise x to the power of y',
 'properties': {'x': {'title': 'X', 'type': 'number'},
  'y': {'title': 'Y', 'type': 'number'}},
 'required': ['x', 'y'],
 'title': 'exponentiate',
 'type': 'object'}

In [6]:
import json

llm_output_string = "{\"x\" : 5, \"y\" : 2}"
llm_output_dict = json.loads(llm_output_string)
llm_output_dict

{'x': 5, 'y': 2}

In [7]:
exponentiate.func(**llm_output_dict)

25

In [8]:
subtract.func(**llm_output_dict)

-3

We need this agent to remember previous interactions within the conversation. To do that, we will use the `ChatPromptTemplate` with a system message, a placeholder for our chat history, a placeholder for the user query, and finally a placeholder for the agent scratchpad.

The agent scratchpad is where the agent writes its notes as it works through multiple internal thought and tool-use steps to produce a final output for the user. This scratchpad is a list of messages with alternating roles of `ai` (for the tool call) and `tool` (for the tool execution output). Both message types require a `tool_call_id` field which is used to link the respective AI and tool messages - this can be required when we many tool calls happening in parallel.

In [9]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages([
    ("system", (
        "You're a helpful assistant. When answering a user's question "
        "you should first use one of the tools provided. After using a "
        "tool the tool output will be provided in the "
        "'scratchpad' below. If you have an answer in the "
        "scratchpad you should not use any more tools and "
        "instead answer directly to the user."
    )),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])

In [16]:
from langchain_core.runnables.base import RunnableSerializable

tools = [add, subtract, multiply, exponentiate]

# define the agent runnable
agent: RunnableSerializable = (
    {
        "input": lambda x: x["input"],
        "chat_history": lambda x: x["chat_history"],
        "agent_scratchpad": lambda x: x.get("agent_scratchpad", [])
    }
    | prompt
    | llm.bind_tools(tools, tool_choice="auto")
)

In [17]:
tool_call = agent.invoke({"input": "What is 10 + 10", "chat_history": []})
tool_call

AIMessage(content='', additional_kwargs={'reasoning_content': 'Okay, the user is asking "What is 10 + 10". Let me see. I need to use the tools provided to answer this. The available functions are add, subtract, multiply, and exponentiate. The question is straightforward addition. So I should use the add function. The parameters are x and y, both 10. Let me make sure the function requires them. The add function\'s parameters are x and y, both numbers. So I\'ll call add with x=10 and y=10. That should give the result 20. I don\'t need any other tools here. Just apply the add function.\n', 'tool_calls': [{'id': 'ata6tdsda', 'function': {'arguments': '{"x":10,"y":10}', 'name': 'add'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 165, 'prompt_tokens': 429, 'total_tokens': 594, 'completion_time': 0.630963996, 'prompt_time': 0.02675479, 'queue_time': 0.126024969, 'total_time': 0.657718786}, 'model_name': 'qwen/qwen3-32b', 'system_fingerprint': 'fp_5cf921caa2'

In [18]:
tool_call.tool_calls

[{'name': 'add',
  'args': {'x': 10, 'y': 10},
  'id': 'ata6tdsda',
  'type': 'tool_call'}]

From here, we have the tool `name` that our LLM wants to use and the `args` that it
wants to pass to that tool. We can see that the tool `add` is being used with the
arguments `x=10` and `y=10`. The `agent.invoke` method has _not_ executed the tool
function; we need to write that part of the agent code ourselves.

Executing the tool code requires two steps:

1. Map the tool `name` to the tool function.

2. Execute the tool function with the generated `args`.

In [19]:
# create tool name to function mapping

name2tool = {tool.name : tool.func for tool in tools}
name2tool

{'add': <function __main__.add(x: float, y: float) -> float>,
 'subtract': <function __main__.subtract(x: float, y: float) -> float>,
 'multiply': <function __main__.multiply(x: float, y: float) -> float>,
 'exponentiate': <function __main__.exponentiate(x: float, y: float) -> float>}

In [20]:
tool_exec_content = name2tool[tool_call.tool_calls[0]["name"]](
    **tool_call.tool_calls[0]["args"]
)

tool_exec_content

20

In [21]:
from langchain_core.messages import ToolMessage

tool_exec = ToolMessage(
    content=f"The {tool_call.tool_calls[0]['name']} tool returned {tool_exec_content}",
    tool_call_id=tool_call.tool_calls[0]["id"]
)

out = agent.invoke({
    "input": "What is 10 + 10",
    "chat_history": [],
    "agent_scratchpad": [tool_call, tool_exec]
})
out

AIMessage(content='The result of 10 + 10 is **20**.', additional_kwargs={'reasoning_content': 'Okay, the user asked "What is 10 + 10". I need to use the add function here. Let me check the tools available. The add function takes two numbers, x and y. So I\'ll call add with x=10 and y=10. The result from the tool was 20. That\'s straightforward. I just need to present the answer clearly. Make sure to state the calculation and the result. No other tools are needed here since it\'s a simple addition. Just confirm the numbers and the operation. Yep, everything checks out. The answer is 20.\n'}, response_metadata={'token_usage': {'completion_tokens': 147, 'prompt_tokens': 476, 'total_tokens': 623, 'completion_time': 0.545555853, 'prompt_time': 0.029417441, 'queue_time': 0.134094188, 'total_time': 0.574973294}, 'model_name': 'qwen/qwen3-32b', 'system_fingerprint': 'fp_5cf921caa2', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--f5fcebc1-fd90-4d7b-99f0-f8fc11

Despite having the answer in our `agent_scratchpad`, the LLM still tries to use the tool
_again_. This behaviour happens because we bonded the tools to the LLM with
`tool_choice="any"`. When we set `tool_choice` to `"any"` or `"required"`, we tell the
LLM that it _MUST_ use a tool, i.e., it cannot provide a final answer.

There's two options to fix this:

1. Set `tool_choice="auto"` to tell the LLM that it can choose to use a tool or provide
a final answer.

2. Create a `final_answer` tool - we'll explain this shortly.

First, let's try option **1**:

In [22]:
agent: RunnableSerializable = (
    {
        "input" : lambda x: x["input"],
        "chat_history" : lambda x: x["chat_history"],
        "agent_scratchpad": lambda x: x.get("agent_scratchpad", [])
    }
    | prompt
    | llm.bind_tools(tools, tool_choice = "auto")

)

In [23]:
tool_call = agent.invoke({"input": "What is 10 + 10", "chat_history": []})
tool_call

AIMessage(content='', additional_kwargs={'reasoning_content': 'Okay, the user is asking "What is 10 + 10". Let me see. I need to use the tools provided to answer this. The available functions are add, subtract, multiply, and exponentiate. The question is about addition, so the add function is the right choice. The parameters required are x and y, both numbers. Here, x is 10 and y is 10. So I should call the add function with these values. Let me double-check the function\'s parameters to make sure. Yep, add takes x and y as numbers. So the tool call should be add with x=10 and y=10. No other functions are needed here. The answer should be 20.\n', 'tool_calls': [{'id': '46gdz36as', 'function': {'arguments': '{"x":10,"y":10}', 'name': 'add'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 183, 'prompt_tokens': 429, 'total_tokens': 612, 'completion_time': 0.323311479, 'prompt_time': 0.028514668, 'queue_time': 0.125377082, 'total_time': 0.351826147}, 'model_

In [24]:
tool_output = name2tool[tool_call.tool_calls[0]["name"]](
    **tool_call.tool_calls[0]["args"]
)

tool_exec = ToolMessage(
    content=f"The {tool_call.tool_calls[0]['name']} tool returned {tool_output}",
    tool_call_id=tool_call.tool_calls[0]["id"]
)

out = agent.invoke({
    "input": "What is 10 + 10",
    "chat_history": [],
    "agent_scratchpad": [tool_call, tool_exec]
})
out

AIMessage(content='The result of 10 + 10 is **20**.', additional_kwargs={'reasoning_content': 'Okay, the user asked "What is 10 + 10". I need to use the add function here. Let me check the tools available. The add function takes two numbers, x and y. So I\'ll call add with x=10 and y=10. The result from the tool was 20. That\'s straightforward. I just need to present the answer clearly. Let me make sure there\'s no trick or additional steps needed. The user probably just wants the sum. So the answer is 20.\n'}, response_metadata={'token_usage': {'completion_tokens': 132, 'prompt_tokens': 476, 'total_tokens': 608, 'completion_time': 0.436795241, 'prompt_time': 0.030679978, 'queue_time': 0.125519582, 'total_time': 0.467475219}, 'model_name': 'qwen/qwen3-32b', 'system_fingerprint': 'fp_5cf921caa2', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--7fb823a5-3dd6-4519-8073-e6191082c77e-0', usage_metadata={'input_tokens': 476, 'output_tokens': 132, 'total_toke

option 2 - final answer tool

In [25]:
@tool
def final_answer(answer: str, tools_used: list[str]) -> str:
    """Use this tool to provide a final answer to the user.
    The answer should be in natural language as this will be provided
    to the user directly. The tools_used must include a list of tool
    names that were used within the `scratchpad`.
    """
    return {"answer": answer, "tools_used": tools_used}

In [26]:
tools = [final_answer, add, subtract, multiply, exponentiate]

# we need to update our name2tool mapping too
name2tool = {tool.name: tool.func for tool in tools}

agent: RunnableSerializable = (
    {
        "input": lambda x: x["input"],
        "chat_history": lambda x: x["chat_history"],
        "agent_scratchpad": lambda x: x.get("agent_scratchpad", [])
    }
    | prompt
    | llm.bind_tools(tools, tool_choice="any")  # we're forcing tool use again
)

In [27]:
tool_call = agent.invoke({"input": "What is 10 + 10", "chat_history": []})
tool_call.tool_calls

[{'name': 'add',
  'args': {'x': 10, 'y': 10},
  'id': 'xgv6c64c4',
  'type': 'tool_call'}]

In [28]:
tool_out = name2tool[tool_call.tool_calls[0]["name"]](
    **tool_call.tool_calls[0]["args"]
)

tool_exec = ToolMessage(
    content=f"The {tool_call.tool_calls[0]['name']} tool returned {tool_out}",
    tool_call_id=tool_call.tool_calls[0]["id"]
)

out = agent.invoke({
    "input": "What is 10 + 10",
    "chat_history": [],
    "agent_scratchpad": [tool_call, tool_exec]
})
out

AIMessage(content='', additional_kwargs={'reasoning_content': 'Okay, the user asked "What is 10 + 10". I used the add function with x=10 and y=10. The result was 20. Now I need to present the final answer. Since the tool already gave the correct result, I just need to state it clearly. Make sure to mention the tools used, which is the add function here. No other tools are needed. Just confirm the calculation and provide the answer in a natural sentence.\n', 'tool_calls': [{'id': '7ea4zmchy', 'function': {'arguments': '{"answer":"The result of 10 + 10 is 20.","tools_used":["add"]}', 'name': 'final_answer'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 159, 'prompt_tokens': 603, 'total_tokens': 762, 'completion_time': 0.436434388, 'prompt_time': 0.036469632, 'queue_time': 0.134849206, 'total_time': 0.47290402}, 'model_name': 'qwen/qwen3-32b', 'system_fingerprint': 'fp_5cf921caa2', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': No

In [29]:
out.tool_calls

[{'name': 'final_answer',
  'args': {'answer': 'The result of 10 + 10 is 20.', 'tools_used': ['add']},
  'id': '7ea4zmchy',
  'type': 'tool_call'}]

In [30]:
out.tool_calls[0]["args"]

{'answer': 'The result of 10 + 10 is 20.', 'tools_used': ['add']}

Building a custom agent execution tool.

In [31]:
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage

class CustomAgentExecutor:
    chat_history: list[BaseMessage]

    def __init__(self, max_iterations : int = 3):
        self.chat_history = []
        self.max_iterations = max_iterations
        self.agent : RunnableSerializable = (
            {
                "input" : lambda x: x["input"],
                "chat_history" : lambda x : x["chat_history"],
                "agent_scratchpad" : lambda x: x.get("agent_scratchpad", [])
            }
            | prompt
            | llm.bind_tools(tools, tool_choice = "any")
        )

    def invoke(self, input: str) -> dict:
        # invoke the agent but we do this iteratively in a loop until
        # reaching the final answer
        count = 0
        agent_scratchpad = []
        while count < self.max_iterations:
            tool_call = self.agent.invoke({
                "input" : input,
                "chat_history" : self.chat_history,
                "agent_scratchpad" : agent_scratchpad
            })
            # add initial tool call to scratchpad
            agent_scratchpad.append(tool_call)
            # otherwise we execute the tool and add it's output
            # to the agent scratchpad.
            tool_name = tool_call.tool_calls[0]["name"]
            tool_args = tool_call.tool_calls[0]["args"]
            tool_call_id = tool_call.tool_calls[0]["id"]
            tool_out = name2tool[tool_name](**tool_args)
            # add the tools output to the agent scratchpad
            tool_exec = ToolMessage(
                content = f"{tool_out}",
                tool_call_id = tool_call_id
            )
            agent_scratchpad.append(tool_exec)
            # add a print so we can see the intermediate steps
            print(f"{count}: {tool_name}({tool_args})")
            count +=1
            # if the tool call is the final answer tool, we stop.
            if tool_name == "final_answer":
                break
        
        # add the final output to the chat history
        final_answer = tool_out["answer"]
        self.chat_history.extend([
            HumanMessage(content = input),
            AIMessage(content = final_answer)
        ])

        # return the answer in dict form
        return json.dumps(tool_out)

In [32]:
agent_executor = CustomAgentExecutor()

In [33]:
agent_executor.invoke(input="What is 10 + 10")

0: add({'x': 10, 'y': 10})
1: final_answer({'answer': 'The result of 10 + 10 is 20.', 'tools_used': ['add']})


'{"answer": "The result of 10 + 10 is 20.", "tools_used": ["add"]}'